In [1]:
import os
import pandas as pd
from tqdm import tqdm
import sys
import logging
import numpy as np
import itertools
import random

sys.path.insert(0, './_modules')
from db_functions import TheDB
from datetime import datetime


In [2]:
def IKYDLS():
    
    ## bots BD connections
    conn_dict = {'host':'localhost', 'database': 'brspcqadev',
                 'user':'root', 'password':'quito2020'}

    cqdb_prod = TheDB(conn_dict)    
    
    
    def CleanList(input_list):
        final_list = [item for item in input_list if item != '' and item != ' ']
        return final_list

    def SetList(input_list):
        final_list = list(set(input_list))
        return final_list
    
    ### working the Screening table
    
    ## getting the table
    df_full = cqdb_prod.GetTable('Screening')

    ## converting to time object
    df_full['mail_hms_received'] = pd.to_datetime(df_full['mail_hms_received']).dt.time

    ## converting to datetime object
    df_full['mail_date_received'] = pd.to_datetime(df_full['mail_date_received'])


    ## unifying date and time columns into one single datetime object:
    df_full['mail_datetime_received'] = pd.to_datetime(df_full['mail_date_received'].apply(str) +\
                                                  ' ' + df_full['mail_hms_received'].apply(str))

    df_full['mail_datetime_received'] = pd.to_datetime(df_full['mail_datetime_received'])

    df_full = df_full.sort_values('mail_datetime_received', ascending = False).reset_index(drop = True)   
    
    
    
    ## case view
    df_case_view = df_full.groupby('case_id')[['main_id', 'external_ref_number', 'thread_id']].agg(list).reset_index()
    df_case_view = df_case_view[(df_case_view['case_id'] != '') & (df_case_view['case_id'] != ' ')]
    df_case_view.columns = ['case_id', 'main_id_list', 'external_ref_number_list', 'thread_id_list']
    df_case_view['main_id_list'] = df_case_view['main_id_list'].apply(CleanList).apply(SetList)
    df_case_view['external_ref_number_list'] = df_case_view['external_ref_number_list'].apply(CleanList).apply(SetList)
    df_case_view['thread_id_list'] = df_case_view['thread_id_list'].apply(CleanList).apply(SetList)
    
    
    
    ## getting case_id - main_id relationship:
    df_case_main = df_case_view.set_index(['case_id'])['main_id_list'].apply(pd.Series)\
                                             .stack()\
                                             .to_frame().reset_index(level=1, drop=True).reset_index()
    df_case_main.columns = ['case_id', 'main_id']
    df_case_main['main_id'] = df_case_main['main_id'].astype(int)
    
    
    
    ## getting the case_id - external_ref_number relationship
    df_case_ext = df_case_view.set_index(['case_id'])['external_ref_number_list'].apply(pd.Series)\
                                         .stack()\
                                         .to_frame().reset_index(level=1, drop=True).reset_index()
    df_case_ext.columns = ['case_id', 'external_ref_number_list']
    
    
    
    ## case_id - thread_id relationship
    df_case_thread = df_case_view.set_index(['case_id'])['thread_id_list'].apply(pd.Series)\
                                             .stack()\
                                             .to_frame().reset_index(level=1, drop=True).reset_index()
    df_case_thread.columns = ['case_id', 'thread_id']    
    
    
    
    ## external ref number view
    df_ex_view = df_full.groupby('external_ref_number')[['main_id', 'case_id', 'thread_id']].agg(list).reset_index()
    df_ex_view = df_ex_view[(df_ex_view['external_ref_number'] != '') & (df_ex_view['external_ref_number'] != ' ')]
    df_ex_view.columns = ['external_ref_number', 'main_id_list', 'case_id_list', 'thread_id_list']
    df_ex_view['main_id_list'] = df_ex_view['main_id_list'].apply(CleanList).apply(SetList)
    df_ex_view['case_id_list'] = df_ex_view['case_id_list'].apply(CleanList).apply(SetList)
    df_ex_view['thread_id_list'] = df_ex_view['thread_id_list'].apply(CleanList).apply(SetList) 
    
    
    ## external_ref_number - main_id relation
    df_ex_main = df_ex_view.set_index(['external_ref_number'])['main_id_list'].apply(pd.Series)\
                                             .stack()\
                                             .to_frame().reset_index(level=1, drop=True).reset_index()
    df_ex_main.columns = ['external_ref_number', 'main_id']
    df_ex_main['main_id'] = df_ex_main['main_id'].astype(int)   
    
    
    ## external_ref_number - case_id relation
    df_ex_case = df_ex_view.set_index(['external_ref_number'])['case_id_list'].apply(pd.Series)\
                                             .stack()\
                                             .to_frame().reset_index(level=1, drop=True).reset_index()
    df_ex_case.columns = ['external_ref_number', 'case_id']    
    
    
    ## external_ref_number - thread_id relation
    df_ex_thread = df_ex_view.set_index(['external_ref_number'])['thread_id_list'].apply(pd.Series)\
                                             .stack()\
                                             .to_frame().reset_index(level=1, drop=True).reset_index()
    df_ex_thread.columns = ['external_ref_number', 'thread_id']    
    
    
    ## trix_id - external_ref_number relation
    df_trix = cqdb_prod.GetTable('Trix')
    df_trix_ext = df_trix.groupby('external_ref_number')[['case_id', 'line_id']].agg(list).reset_index()
    df_trix_ext = df_trix_ext.rename(columns = {'line_id': 'line_id_list'})
    df_trix_ext = df_trix_ext.loc[:, ['external_ref_number', 'line_id_list']]    
    
    
    
    ## trix_id - external_ref_number relation
    df_trix_ext2 = df_trix_ext.set_index(['external_ref_number'])['line_id_list'].apply(pd.Series)\
                                             .stack()\
                                             .to_frame().reset_index(level=1, drop=True).reset_index()
    df_trix_ext2.columns = ['external_ref_number', 'line_id']
    df_trix_ext2['line_id'] = df_trix_ext2['line_id'].astype(int)
    
    
    ## trix_id - case_id relation
    df_trix_case = df_trix.groupby('case_id')[['external_ref_number', 'line_id']].agg(list).reset_index()
    df_trix_case = df_trix_case.rename(columns = {'line_id': 'line_id_list'})
    df_trix_case = df_trix_case.loc[:, ['case_id', 'line_id_list']]    
    
    
    df_trix_case2 = df_trix_case.set_index(['case_id'])['line_id_list'].apply(pd.Series)\
                                             .stack()\
                                             .to_frame().reset_index(level=1, drop=True).reset_index()
    df_trix_case2.columns = ['case_id', 'line_id']
    df_trix_case2['line_id'] = df_trix_case2['line_id'].astype(int)
    
    
    result_dict = {'case_main': df_case_main, 'case_ext': df_case_ext, 'case_thread': df_case_thread,\
                  'ext_main':df_ex_main, 'ext_case': df_ex_case, 'ext_thread':df_ex_thread, \
                  'trix_ext': df_trix_ext2, 'trix_case': df_trix_case2}
    
    
    
    
    ## Writing data to DB:
    for relation, df in result_dict.items():
        try:
            #print(relation)
            results_db = cqdb_prod.InsertDB(df, 'relation_' + relation, key_column = '' ,\
                          return_table = False, append_table = False)


            #print(' ')
            #print(' ')
        except Exception as e:
            #print('Some error ocurred!')
            logging.error('Linked Cases Robot error: {}'.format(e))
            pass    
    
    
    
    
    return result_dict

In [3]:
result_dict = IKYDLS()

Original length: 34685
Final length: 11431
Success! Data writtten in db!
Original length: 8530
Final length: 2901
Success! Data writtten in db!
Original length: 20870
Final length: 7219
Success! Data writtten in db!
Original length: 27478
Final length: 9696
Success! Data writtten in db!
Original length: 8530
Final length: 2901
Success! Data writtten in db!
Original length: 16383
Final length: 5505
Success! Data writtten in db!
Original length: 10565
Final length: 34440
Success! Data writtten in db!
Original length: 10569
Final length: 14404
Success! Data writtten in db!


In [4]:
df_trix_case = result_dict['trix_case']
df_grouped_trix_case = df_trix_case[df_trix_case['case_id'] != ''].groupby('case_id')['line_id'].\
apply(list).to_frame().reset_index()
df_grouped_trix_case['counting'] = df_grouped_trix_case['line_id'].apply(len)

In [5]:
df_grouped_trix_case[df_grouped_trix_case['counting'] > 1]

,case_id,line_id,counting
4,0-0086000003636,"[51, 86]",2
41,0-0642000025642,"[4990, 5298, 5434]",3
149,0-2440000030192,"[8806, 8866]",2
254,0-4674000029946,"[8700, 8701]",2
305,0-5507000026511,"[5741, 6196]",2
...,...,...,...
5293,9-8451000027238,"[6174, 6800]",2
5324,9-8937000025835,"[4849, 5040]",2
5367,9-9759000030906,"[9209, 9700]",2
5379,9-9949000026755,"[5811, 6187]",2


In [6]:
df_case_ext = result_dict['case_ext']
df_grouped_case_ext = df_case_ext[df_case_ext['case_id'] != ''].groupby('case_id')['external_ref_number_list'].\
apply(list).to_frame().reset_index()
df_grouped_case_ext['counting'] = df_grouped_case_ext['external_ref_number_list'].apply(len)

In [7]:
df_grouped_case_ext[df_grouped_case_ext['counting'] > 1]

,case_id,external_ref_number_list,counting
13,0-0167000030056,"[150047-48.2019.8.26.0077, 1500477-48.2019.8.2...",2
50,0-0532000031324,"[0002033-91.2020.8.21.0004, 004/2.20.0001627-2]",2
73,0-0835000028026,"[00866473-63.2019.8.21.0001, 001/2.19.0054230-5]",2
101,0-1130000030792,"[00569-19.2020.8.16.0024, 0005690-19.2020.8.16...",2
108,0-1173000029761,"[0161961-24.2019.8.21.0001, 001/2.19.0114014-6]",2
...,...,...,...
7885,9-9466000028161,"[0016570-40.2019.8.24.0038, 0016570-40.20198.2...",2
7897,9-9662000031159,"[048.844.525-69, 0007016-14.2020.8.25.0040]",2
7903,9-9736000028248,"[0000436-91.2019.815.0941, 0000436-91.2019.8.1...",2
7906,9-9772000027727,"[00866473-63.2019.8.21.0001, 001/2.19.0054230-5]",2


In [27]:
df_case_main = result_dict['case_main']
df_grouped_case_main = df_case_main[df_case_main['case_id'] != ''].groupby('case_id')['main_id'].\
apply(list).to_frame().reset_index()
df_grouped_case_main['counting'] = df_grouped_case_main['main_id'].apply(len)

In [29]:
df_grouped_case_main

,case_id,main_id,counting
0,0-0021000030705,"[27410, 27603]",2
1,0-0027000028709,"[9573, 17317, 13011]",3
2,0-0035000030501,"[27084, 26894]",2
3,0-0045000030991,"[27768, 27380, 27806]",3
4,0-0046000030816,"[24979, 25101]",2
...,...,...,...
6003,9-9923000030752,"[9371, 3124]",2
6004,9-9939000027087,[12672],1
6005,9-9983000030877,"[25216, 23799, 24000]",3
6006,9-9987000028284,"[7985, 10334]",2


In [25]:
df_case_thread = result_dict['case_thread']
df_grouped_case_thread = df_case_thread[df_case_thread['case_id'] != ''].groupby('case_id')['thread_id'].\
apply(list).to_frame().reset_index()
df_grouped_case_thread['counting'] = df_grouped_case_thread['thread_id'].apply(len)

In [26]:
df_grouped_case_thread[df_grouped_case_thread['counting'] > 1]

,case_id,thread_id,counting
0,0-0021000030705,"[17425c95e7fe841c, 17431c03ffd3c8d0]",2
1,0-0027000028709,"[16f6cdb6e830fc91, 16f823855f9e9e00]",2
2,0-0035000030501,"[1740e732ca3caa81, 17406aa06d2ac174]",2
4,0-0046000030816,"[1739b58c6c95abf7, 1739685f1915e37e]",2
5,0-0054000030714,"[173150c3b9618f6d, 1739114c09720d78, 1737c582a...",4
...,...,...,...
6000,9-9896000027544,"[16d21df5b0a07fc5, 16d27983f7394e29]",2
6001,9-9903000029263,"[1700c69239c4b4d8, 17010fbd0e802be3]",2
6002,9-9913000030750,"[1739bae54e4a50e4, 173932b60a905a0e]",2
6003,9-9923000030752,"[17214eb4c7c1d7e5, 1721a7a4e9ae3c9b]",2


In [58]:
df_trix_ext = result_dict['trix_ext']
df_grouped_trix_ext = df_trix_ext[df_trix_ext['external_ref_number'] != ''].groupby('external_ref_number')['line_id'].\
apply(list).to_frame().reset_index()
df_grouped_trix_ext['counting'] = df_grouped_trix_ext['line_id'].apply(len)

In [59]:
df_grouped_trix_ext

,external_ref_number,line_id,counting
0,0000001-06.2019.8.24.0218,"[4865, 4952, 4953, 4997, 5087]",5
1,0000002-73.2018.4.01.4300,"[2987, 4069]",2
2,0000003-25.2016.4.05.8501,[1044],1
3,0000006-80.2016.4.05.8500,[1163],1
4,0000009-03.2018.4.05.8100,[3295],1
...,...,...,...
4247,9344-13.2018.4.01.3200,[4101],1
4248,9366-80.2019.811,[7168],1
4249,950-71.2016.4.05.0000,[1502],1
4250,9586-35.2019.4.01.3200,[7096],1


In [34]:
df_final = df_grouped_trix_case.merge(df_grouped_case_ext, on = 'case_id',\
                           how = 'inner', suffixes = ('_1', '_2')).merge(df_grouped_case_main,\
                                                on = 'case_id',\
                                                how = 'inner').merge(df_grouped_case_thread,\
                                                                     on = 'case_id', how = 'inner')

In [40]:
df_example = df_final[(df_final['counting_1'] > 2)]
df_example

,case_id,line_id,counting_1,external_ref_number_list,counting_2,main_id,counting_x,thread_id,counting_y
237,1-8246000025746,"[5240, 6173, 6297]",3,[0175523-54.2018.8.06.0001],1,"[1495, 7332, 1412, 20068, 135, 18547, 12471, 1...",9,"[16c052b9e81b9758, 16c01fd59772f211, 16c19753f...",5


In [47]:
df_example['line_id'].values.tolist()

[[5240, 6173, 6297]]

In [52]:
df_test = df_example.loc[:, ['case_id', 'thread_id']]
#df_test = df_test.rename(columns = {'external_ref_number_list': 'external_ref_number'})
df_test.explode('thread_id')

,case_id,thread_id
237,1-8246000025746,16c052b9e81b9758
237,1-8246000025746,16c01fd59772f211
237,1-8246000025746,16c19753ff829709
237,1-8246000025746,16c05f13b233b059
237,1-8246000025746,16c448a428fdda01


In [62]:
df_grouped_trix_ext[df_grouped_trix_ext['external_ref_number'] ==\
                    '0175523-54.2018.8.06.0001'].loc[:, ['external_ref_number', 'line_id']].explode('line_id')

,external_ref_number,line_id
2506,0175523-54.2018.8.06.0001,5240
2506,0175523-54.2018.8.06.0001,6173
2506,0175523-54.2018.8.06.0001,6297
2506,0175523-54.2018.8.06.0001,6850
